## Pregunta 2
### Jorge Schenke Larraín.
### n° de alumno: 17641624

#### Importamos librerías

In [1]:
import random
import math

#### Primalidad

In [2]:
first_primes_list = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29,
                    31, 37, 41, 43, 47, 53, 59, 61, 67,
                    71, 73, 79, 83, 89, 97, 101, 103,
                    107, 109, 113, 127, 131, 137, 139,
                    149, 151, 157, 163, 167, 173, 179,
                    181, 191, 193, 197, 199, 211, 223,
                    227, 229, 233, 239, 241, 251, 257,
                    263, 269, 271, 277, 281, 283, 293,
                    307, 311, 313, 317, 331, 337, 347, 349]                #Primeros 100 primos para confirmar que un número es candidato a primo

In [3]:
def isMillerRabinPassed(miller_rabin_candidate):                 #Test de primalidad de Miller-Rabin solo se correrá si no es divisible por ninguno de los primeros 100 primos
    maxDivisionsByTwo = 0
    evenComponent = miller_rabin_candidate-1
    while evenComponent % 2 == 0:
        evenComponent >>= 1
        maxDivisionsByTwo += 1
    assert(2**maxDivisionsByTwo * evenComponent == miller_rabin_candidate-1)
    def trialComposite(round_tester):
        if pow(round_tester, evenComponent, 
                miller_rabin_candidate) == 1:
            return False
        for i in range(maxDivisionsByTwo):
            if pow(round_tester, 2**i * evenComponent, miller_rabin_candidate) == miller_rabin_candidate-1:
                return False
        return True
    numberOfRabinTrials = 20
    for i in range(numberOfRabinTrials):
        round_tester = random.randrange(2, miller_rabin_candidate)
        if trialComposite(round_tester):
            return False
    return True

In [4]:
def is_prime( n):
    for prime in first_primes_list:
        if n % prime == 0:
            return False                #Si es divisible por alguno de los 1ros 100 primos, no es primo
    if isMillerRabinPassed(n):                #Test de primalidad no asegura primalidad pero es altamente probable
        return True

#### MCD

In [5]:
def gcd1(x, y):        #MCD
    if(y==0):
        return x
    else:
        return gcd1(y,x%y)

#### Inverso modular

In [6]:
def alg_ext_euclides(a: int, b: int):        #Algoritmo extendido de Euclides para encontrar inversos modulares
    r_0 = a
    s_0 = 1
    t_0 = 0
    r_1 = b
    s_1 = 0
    t_1 = 1
    while r_1 > 0:
        r_2 = r_0 % r_1
        s_2 = s_0 - (r_0 // r_1) * s_1
        t_2 = t_0 - (r_0 // r_1) * t_1
        r_0 = r_1
        s_0 = s_1
        t_0 = t_1
        r_1 = r_2
        s_1 = s_2
        t_1 = t_2
    return r_0, s_0, t_0

def inverso(a: int, n: int) -> int:
    (r, s, t) = alg_ext_euclides(a, n)
    return s % n

### RSAReceiver

In [7]:
class RSAReceiver:
    def __init__(self, bit_len):
        self.bit_len = bit_len
        self.p = self.generate_prime()    #P
        self.q = self.generate_prime()    #Q
        self.n = self.p * self.q    #N
        self.phin = (self.p-1) * (self.q-1)    #Phi(N)
        self.d = self.generate_d()    #D
        self.e = self.generate_e()    #E

    def generate_prime(self):
        while True:
            p = random.randint(2**((self.bit_len//2 + 1)-1), 2**(self.bit_len//2 + 1))    #Generamos un número al azar de len_n/2 +1 largo en bi
            if is_prime(p):    #Revisamos si es primo
                return p

    def generate_d(self):
        while True:
            d = random.randint(2**((self.bit_len//2 + 1)-1), 2**(self.bit_len//2 + 1))    #Generamos un número al azar de len_n/2 +1 largo en bits
            if gcd1(d, self.phin) == 1:    #Revisamos si es coprimo de Phi(N)
                return d

    def generate_e(self):
        e = inverso(self.d, self.phin)    #Calculamos inverso modular de D y asignaos a E
        return e

    def get_public_key(self):    #Concatena E y N según especificación para crear llave pública
        len_e = len(bin(self.e)) // 8
        if len(bin(self.e)) % 8 != 0:
            len_e += 1
        e_bytes = bytearray((self.e).to_bytes(len_e, 'big'))
        e_len = len(e_bytes)
        e_len_bytes = e_len.to_bytes(4, 'big')
        len_n = len(bin(self.n)) // 8
        if len(bin(self.n)) % 8 != 0:
            len_n += 1
        n_bytes = bytearray((self.n).to_bytes(len_n, 'big'))
        n_len = len(n_bytes)
        n_len_bytes = n_len.to_bytes(4, 'big')
        public_key = e_len_bytes + e_bytes + n_len_bytes + n_bytes
        return public_key

    def decrypt(self, ciphertext):
        string = []    #Creamos lista vacía de strings
        len_n = len(bin(self.n))    #Calculamos largo de los bloques
        n = 1
        while n*8 < len_n:
            n += 1
        len_c = len(ciphertext)    #Calculamos largo del cifrado
        n_blocks = len_c // n    #Calculamos cantidad de bloques
        for i in range(n_blocks):    #Por cada bloque:
            block = ciphertext[i*n:(i+1)*n]
            block_int = int.from_bytes(block, 'big')    #Lo pasamos a int
            block_int = pow(block_int, self.d, self.n)    #Calculamos int ^ d % n
            msg_block_bytes = block_int.to_bytes(n, 'big')    #Pasamos entero a bytes
            msg_block = msg_block_bytes.decode('utf-8')    #Decodificamos a string
            string.append(msg_block)    #Agregamos string a lista
        msg = ''.join(string)    #Unimos strings
        return msg

### RSASender

In [8]:
class RSASender:
    def __init__(self, public_key):
        self.public_key = public_key
        self.e, self.n = self.get_e_n()
        self.bit_len = math.ceil(math.log(self.n)/math.log(2))
    
    def get_e_n(self):
        e_len_bytes = self.public_key[0:4]    #Buscamos largo de e en bytes
        e_len = int.from_bytes(e_len_bytes, 'big')    #Pasamos largo a int
        e_bytes = self.public_key[4:4+e_len]    #Según el largo, extraemos los bytes de e
        e = int.from_bytes(e_bytes, 'big')    #Pasamos e a int
        n_len_bytes = self.public_key[4+e_len:8+e_len]    #Largo de n en bytes
        n_len = int.from_bytes(n_len_bytes, 'big')    #Largo de n a int
        n_bytes = self.public_key[8+e_len:8+e_len+n_len]    #n en bytes
        n = int.from_bytes(n_bytes, 'big')    #n en int
        return e, n
    
    def encrypt(self, msg):
        len_n = len(bin(self.n))    #Calculamos largo de n en bits
        n = 1
        while n*8 < len_n:    #Calculamos largo del bloque
            n += 1
        n -= 1
        msg_array = bytearray()    #Inicializamos array de bytes vacío
        msg_bytes = bytearray(msg.encode('utf-8'))    #Codificamos el mensaje en UTF-8
        msg_len = len(msg_bytes)    #Calculamos largo del ensaje en bytes
        n_blocks = msg_len // n    #Calculamos número de bloques
        if msg_len % n != 0:
            n_blocks += 1
        for i in range(n_blocks):    #Para cada bloque:
            msg_block = msg_bytes[i*n:(i+1)*n]    #Extraemos el bloque del mensaje
            msg_int = int.from_bytes(msg_block, 'big')    #Pasamos el bloque a int
            cipher_int = pow(msg_int, self.e, self.n)    #Calculamos m ^ e % n
            cipher_bytes = cipher_int.to_bytes(n + 1, 'big')    #Pasamos el cifrado a bytes
            msg_array += cipher_bytes    #Agregamos a lista de bytes
        return msg_array

#### Ejemplo

In [9]:
Rec = RSAReceiver(2048)
Enc = RSASender(Rec.get_public_key())
cipher = Enc.encrypt('Being open source means anyone can independently review '
    'the code. If it was closed source, nobody could verify the '
    'security. I think it’s essential for a program of this '
    'nature to be open source.')
print(f"Mensaje secreto: {Rec.decrypt(cipher)}")

Mensaje secreto:                                                             Being open source means anyone can independently review the code. If it was closed source, nobody could verify the security. I think it’s essential for a program of this nature to be open source.
